# Natural Gas Storage Contract Pricing Model
    Build a prototype pricing model for a natural gas storage contract that the trading desk can use to quote clients who want to buy gas now, store it, and sell it later at higher prices.

    1. Client believes winter will be colder than expected
    2. They want to buy gas now, store it, and sell in winter to profit from
      price increases
    3. Desk needs a pricing tool with manual oversight before full automation

        Core pricing formula : Contract Value = Sale Revenue - Purchase Cost - All Storage/Operating Costs      

##  Import Libraries

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Load Datasets

In [81]:
df = pd.read_csv('Nat_Gas.csv')

### Clean the datasets

In [82]:
df['Dates'] = pd.to_datetime(df['Dates'])
df['Prices'] = df['Prices'].astype(float)

/tmp/ipython-input-2997517366.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dates'] = pd.to_datetime(df['Dates'])


In [83]:
print("Natural Gas Price Data Loaded Successfully!")
print(f"Shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nDate Range: {df['Dates'].min()} to {df['Dates'].max()}")
print(f"Price Range: ${df['Prices'].min():.2f} to ${df['Prices'].max():.2f}")

Natural Gas Price Data Loaded Successfully!
Shape: (48, 2)

First few rows:
       Dates  Prices
0 2020-10-31    10.1
1 2020-11-30    10.3
2 2020-12-31    11.0
3 2021-01-31    10.9
4 2021-02-28    10.9

Date Range: 2020-10-31 00:00:00 to 2024-09-30 00:00:00
Price Range: $9.84 to $12.80


##  DATA EXPLORATION

###Understanding the data before building the model

In [84]:
# Basic statistics
print("\n1. DESCRIPTIVE STATISTICS:")
print("-" * 40)
print(df['Prices'].describe())


1. DESCRIPTIVE STATISTICS:
----------------------------------------
count    48.000000
mean     11.207083
std       0.757897
min       9.840000
25%      10.650000
50%      11.300000
75%      11.625000
max      12.800000
Name: Prices, dtype: float64


In [85]:
# Check for missing values
print("\n2. DATA QUALITY CHECK:")
print("-" * 40)
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Total records: {len(df)}")


2. DATA QUALITY CHECK:
----------------------------------------
Missing values: 0
Total records: 48


In [86]:
# Add month and year columns for seasonal analysis
df['Year'] = df['Dates'].dt.year
df['Month'] = df['Dates'].dt.month

In [87]:
# Monthly average prices
print("\n3. MONTHLY AVERAGE PRICES:")
print("-" * 40)
monthly_avg = df.groupby('Month')['Prices'].mean().round(2)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month_num, month_name in enumerate(months, 1):
    print(f"{month_name}: ${monthly_avg[month_num]:.2f}")


3. MONTHLY AVERAGE PRICES:
----------------------------------------
Jan: $11.78
Feb: $11.70
Mar: $11.78
Apr: $11.18
May: $10.78
Jun: $10.70
Jul: $10.90
Aug: $10.82
Sep: $11.08
Oct: $10.75
Nov: $11.32
Dec: $11.70


In [88]:
# Identify summer (cheap) vs winter (expensive) periods
summer_months = [5, 6, 7, 8]  # May to August
winter_months = [12, 1, 2]     # December to February

summer_avg = df[df['Month'].isin(summer_months)]['Prices'].mean()
winter_avg = df[df['Month'].isin(winter_months)]['Prices'].mean()

In [89]:
print("\n4. SEASONAL PATTERN:")
print("-" * 40)
print(f"Summer Average (May-Aug): ${summer_avg:.2f}")
print(f"Winter Average (Dec-Feb): ${winter_avg:.2f}")
print(f"Winter Premium: ${winter_avg - summer_avg:.2f} ({((winter_avg/summer_avg - 1)*100):.1f}%)")

print("\n5. KEY INSIGHT:")
print("-" * 40)
print("✓ Winter prices are higher than summer prices")
print("✓ This supports the storage strategy: buy low in summer, sell high in winter")
print("✓ Next step: Build the pricing model to calculate if it's profitable!")


4. SEASONAL PATTERN:
----------------------------------------
Summer Average (May-Aug): $10.80
Winter Average (Dec-Feb): $11.73
Winter Premium: $0.92 (8.5%)

5. KEY INSIGHT:
----------------------------------------
✓ Winter prices are higher than summer prices
✓ This supports the storage strategy: buy low in summer, sell high in winter
✓ Next step: Build the pricing model to calculate if it's profitable!


##  BASIC PRICING FUNCTION
     Build a simple function to calculate gross profit
     Formula: Gross Profit = (Sell Price - Buy Price) × Volume

Calculate gross profit from storage contract
    
    Parameters:
    -----------
    buy_price : float
        Price per MMBtu when buying/injecting gas
    sell_price : float
        Price per MMBtu when selling/withdrawing gas
    volume : float
        Volume of gas in MMBtu
    
    Returns:
    --------
    float : Gross profit in dollars

In [90]:
def calculate_gross_profit(buy_price, sell_price, volume):
    """
    Calculate gross profit from storage contract
    Formula: Gross Profit = (Sell Price - Buy Price) × Volume
    """
    price_difference = sell_price - buy_price
    gross_profit = price_difference * volume
    return gross_profit


# Test
buy_price = 11.00
sell_price = 12.50
volume = 1_000_000

gross_profit = calculate_gross_profit(buy_price, sell_price, volume)

print(f"\nTest Example:")
print(f"  Buy: ${buy_price:.2f}, Sell: ${sell_price:.2f}, Volume: {volume:,.0f} MMBtu")
print(f"  Gross Profit: ${gross_profit:,.2f}")



Test Example:
  Buy: $11.00, Sell: $12.50, Volume: 1,000,000 MMBtu
  Gross Profit: $1,500,000.00


## Storage Costs Added
  Function to calculate storage costs (monthly rate × months)
  
  Updated net profit function that subtracts storage costs

In [91]:
def calculate_storage_cost(storage_rate_per_month, num_months):
    """
    Calculate total storage cost
    Formula: Storage Cost = Monthly Rate × Number of Months
    """
    total_storage_cost = storage_rate_per_month * num_months
    return total_storage_cost


def calculate_net_profit(buy_price, sell_price, volume, storage_rate_per_month, num_months):
    """
    Calculate net profit after storage costs
    """
    gross_profit = calculate_gross_profit(buy_price, sell_price, volume)
    storage_cost = calculate_storage_cost(storage_rate_per_month, num_months)
    net_profit = gross_profit - storage_cost
    return net_profit, gross_profit, storage_cost


# Test
storage_rate = 100_000
num_months = 6

net_profit, gross_profit, storage_cost = calculate_net_profit(
    buy_price, sell_price, volume, storage_rate, num_months
)

In [92]:
print(f"\nTest with Storage Costs:")
print(f"  Gross Profit: ${gross_profit:,.2f}")
print(f"  Storage Cost: ${storage_cost:,.2f} ({num_months} months)")
print(f"  Net Profit: ${net_profit:,.2f}")


Test with Storage Costs:
  Gross Profit: $1,500,000.00
  Storage Cost: $600,000.00 (6 months)
  Net Profit: $900,000.00


## Add Injection/Withdrawal Costs
    Injection cost function ($0.01 × 1M = $10,000)

    Withdrawal cost function ($0.01 × 1M = $10,000)
    
    Complete profit calculation with breakdown:

In [93]:
def calculate_injection_withdrawal_costs(volume, injection_rate, withdrawal_rate):
    """
    Calculate injection and withdrawal costs
    """
    injection_cost = volume * injection_rate
    withdrawal_cost = volume * withdrawal_rate
    total_cost = injection_cost + withdrawal_cost
    return injection_cost, withdrawal_cost, total_cost


def calculate_total_profit(buy_price, sell_price, volume,
                           storage_rate_per_month, num_months,
                           injection_rate, withdrawal_rate):
    """
    Calculate total profit after all operational costs
    """
    gross_profit = calculate_gross_profit(buy_price, sell_price, volume)
    storage_cost = calculate_storage_cost(storage_rate_per_month, num_months)
    injection_cost, withdrawal_cost, inj_with_total = calculate_injection_withdrawal_costs(
        volume, injection_rate, withdrawal_rate
    )
    total_profit = gross_profit - storage_cost - inj_with_total

    return {
        'gross_profit': gross_profit,
        'storage_cost': storage_cost,
        'injection_cost': injection_cost,
        'withdrawal_cost': withdrawal_cost,
        'total_costs': storage_cost + inj_with_total,
        'total_profit': total_profit
    }


# Test
injection_rate = 0.01
withdrawal_rate = 0.01

result = calculate_total_profit(
    buy_price, sell_price, volume,
    storage_rate, num_months,
    injection_rate, withdrawal_rate
)

In [94]:
print(f"\nTest with Injection/Withdrawal Costs:")
print(f"  Gross Profit: ${result['gross_profit']:,.2f}")
print(f"  Storage Cost: ${result['storage_cost']:,.2f}")
print(f"  Injection Cost: ${result['injection_cost']:,.2f}")
print(f"  Withdrawal Cost: ${result['withdrawal_cost']:,.2f}")
print(f"  Total Profit: ${result['total_profit']:,.2f}")



Test with Injection/Withdrawal Costs:
  Gross Profit: $1,500,000.00
  Storage Cost: $600,000.00
  Injection Cost: $10,000.00
  Withdrawal Cost: $10,000.00
  Total Profit: $880,000.00


## Add Transportation Costs

In [95]:
def calculate_transport_costs(transport_cost_in, transport_cost_out):
    """
    Calculate total transportation costs
    """
    total_transport_cost = transport_cost_in + transport_cost_out
    return total_transport_cost


def calculate_final_contract_value(buy_price, sell_price, volume,
                                   storage_rate_per_month, num_months,
                                   injection_rate, withdrawal_rate,
                                   transport_cost_in, transport_cost_out):
    """
    Calculate final contract value with ALL costs included
    """
    gross_profit = calculate_gross_profit(buy_price, sell_price, volume)
    storage_cost = calculate_storage_cost(storage_rate_per_month, num_months)
    injection_cost, withdrawal_cost, inj_with_total = calculate_injection_withdrawal_costs(
        volume, injection_rate, withdrawal_rate
    )
    transport_cost = calculate_transport_costs(transport_cost_in, transport_cost_out)

    total_costs = storage_cost + inj_with_total + transport_cost
    contract_value = gross_profit - total_costs

    return {
        'gross_profit': gross_profit,
        'storage_cost': storage_cost,
        'injection_cost': injection_cost,
        'withdrawal_cost': withdrawal_cost,
        'transport_cost_in': transport_cost_in,
        'transport_cost_out': transport_cost_out,
        'total_transport_cost': transport_cost,
        'total_costs': total_costs,
        'contract_value': contract_value
    }


# Test
transport_in = 50_000
transport_out = 50_000

result = calculate_final_contract_value(
    buy_price, sell_price, volume,
    storage_rate, num_months,
    injection_rate, withdrawal_rate,
    transport_in, transport_out
)

In [96]:
print(f"\nComplete Contract Valuation:")
print(f"  Gross Profit: ${result['gross_profit']:,.2f}")
print(f"  Total Costs: ${result['total_costs']:,.2f}")
print(f"  Contract Value: ${result['contract_value']:,.2f}")


Complete Contract Valuation:
  Gross Profit: $1,500,000.00
  Total Costs: $720,000.00
  Contract Value: $780,000.00


## Full Multi-Date Pricing Model
      HANDLE MULTIPLE DATES WITH REAL PRICE DATA



In [97]:
def get_price_for_date(df, date):
    """
    Get price for a specific date from the dataframe
    """
    target_date = pd.to_datetime(date)
    df_sorted = df.sort_values('Dates')
    idx = (df_sorted['Dates'] - target_date).abs().idxmin()
    price = df_sorted.loc[idx, 'Prices']
    actual_date = df_sorted.loc[idx, 'Dates']
    return price, actual_date


def calculate_contract_value_multi_dates(
    df,
    injection_dates,
    injection_volumes,
    withdrawal_dates,
    withdrawal_volumes,
    storage_rate_per_month,
    injection_rate_per_mmbtu,
    withdrawal_rate_per_mmbtu,
    transport_cost_per_injection,
    transport_cost_per_withdrawal
):
    """
    Calculate contract value with multiple injection/withdrawal dates
    This is the MAIN PRICING FUNCTION for production use
    """

    # Get prices for all injection dates
    injection_details = []
    total_purchase_cost = 0
    for date, volume in zip(injection_dates, injection_volumes):
        price, actual_date = get_price_for_date(df, date)
        cost = price * volume
        total_purchase_cost += cost
        injection_details.append({
            'date': date,
            'actual_date': actual_date,
            'price': price,
            'volume': volume,
            'cost': cost
        })

    # Get prices for all withdrawal dates
    withdrawal_details = []
    total_sale_revenue = 0
    for date, volume in zip(withdrawal_dates, withdrawal_volumes):
        price, actual_date = get_price_for_date(df, date)
        revenue = price * volume
        total_sale_revenue += revenue
        withdrawal_details.append({
            'date': date,
            'actual_date': actual_date,
            'price': price,
            'volume': volume,
            'revenue': revenue
        })

    # Calculate gross profit
    gross_profit = total_sale_revenue - total_purchase_cost

    # Calculate storage duration
    all_dates = [pd.to_datetime(d) for d in injection_dates + withdrawal_dates]
    start_date = min(all_dates)
    end_date = max(all_dates)
    num_months = ((end_date.year - start_date.year) * 12 +
                  (end_date.month - start_date.month) + 1)
    storage_cost = storage_rate_per_month * num_months

    # Calculate injection/withdrawal costs
    total_volume_injected = sum(injection_volumes)
    total_volume_withdrawn = sum(withdrawal_volumes)
    injection_cost = total_volume_injected * injection_rate_per_mmbtu
    withdrawal_cost = total_volume_withdrawn * withdrawal_rate_per_mmbtu

    # Calculate transportation costs
    num_injections = len(injection_dates)
    num_withdrawals = len(withdrawal_dates)
    transport_cost = (num_injections * transport_cost_per_injection +
                     num_withdrawals * transport_cost_per_withdrawal)

    # Calculate total costs and contract value
    total_costs = storage_cost + injection_cost + withdrawal_cost + transport_cost
    contract_value = gross_profit - total_costs

    return {
        'injection_details': injection_details,
        'withdrawal_details': withdrawal_details,
        'total_purchase_cost': total_purchase_cost,
        'total_sale_revenue': total_sale_revenue,
        'gross_profit': gross_profit,
        'storage_months': num_months,
        'storage_cost': storage_cost,
        'injection_cost': injection_cost,
        'withdrawal_cost': withdrawal_cost,
        'transport_cost': transport_cost,
        'total_costs': total_costs,
        'contract_value': contract_value,
        'total_volume': total_volume_injected
    }

###FINAL TEST: MULTIPLE INJECTION/WITHDRAWAL DATES

In [98]:
# Scenario: Buy gas in summer months, sell in winter months
injection_dates = ['2023-05-31', '2023-06-30', '2023-07-31']
injection_volumes = [300_000, 400_000, 300_000]  # Total 1M MMBtu

withdrawal_dates = ['2023-11-30', '2023-12-31', '2024-01-31']
withdrawal_volumes = [300_000, 400_000, 300_000]  # Total 1M MMBtu

result = calculate_contract_value_multi_dates(
    df,
    injection_dates,
    injection_volumes,
    withdrawal_dates,
    withdrawal_volumes,
    storage_rate_per_month=100_000,
    injection_rate_per_mmbtu=0.01,
    withdrawal_rate_per_mmbtu=0.01,
    transport_cost_per_injection=50_000,
    transport_cost_per_withdrawal=50_000
)


In [99]:
print("\nINJECTION SCHEDULE:")
for detail in result['injection_details']:
    print(f"  {detail['date']}: {detail['volume']:>10,.0f} MMBtu @ ${detail['price']:.2f} = ${detail['cost']:>12,.2f}")

print("\nWITHDRAWAL SCHEDULE:")
for detail in result['withdrawal_details']:
    print(f"  {detail['date']}: {detail['volume']:>10,.0f} MMBtu @ ${detail['price']:.2f} = ${detail['revenue']:>12,.2f}")



INJECTION SCHEDULE:
  2023-05-31:    300,000 MMBtu @ $11.20 = $3,360,000.00
  2023-06-30:    400,000 MMBtu @ $10.90 = $4,360,000.00
  2023-07-31:    300,000 MMBtu @ $11.40 = $3,420,000.00

WITHDRAWAL SCHEDULE:
  2023-11-30:    300,000 MMBtu @ $12.20 = $3,660,000.00
  2023-12-31:    400,000 MMBtu @ $12.80 = $5,120,000.00
  2024-01-31:    300,000 MMBtu @ $12.60 = $3,780,000.00


In [100]:
print(f"\n{'='*70}")
print("CONTRACT VALUATION SUMMARY:")
print(f"{'='*70}")
print(f"Total Purchase Cost:         ${result['total_purchase_cost']:>15,.2f}")
print(f"Total Sale Revenue:          ${result['total_sale_revenue']:>15,.2f}")
print(f"Gross Profit:                ${result['gross_profit']:>15,.2f}")
print(f"\nStorage ({result['storage_months']} months):           ${result['storage_cost']:>15,.2f}")
print(f"Injection Costs:             ${result['injection_cost']:>15,.2f}")
print(f"Withdrawal Costs:            ${result['withdrawal_cost']:>15,.2f}")
print(f"Transportation Costs:        ${result['transport_cost']:>15,.2f}")
print(f"{'-'*70}")
print(f"Total Costs:                 ${result['total_costs']:>15,.2f}")
print(f"{'='*70}")
print(f"FINAL CONTRACT VALUE:        ${result['contract_value']:>15,.2f}")
print(f"{'='*70}")


CONTRACT VALUATION SUMMARY:
Total Purchase Cost:         $  11,140,000.00
Total Sale Revenue:          $  12,560,000.00
Gross Profit:                $   1,420,000.00

Storage (9 months):           $     900,000.00
Injection Costs:             $      10,000.00
Withdrawal Costs:            $      10,000.00
Transportation Costs:        $     300,000.00
----------------------------------------------------------------------
Total Costs:                 $   1,220,000.00
FINAL CONTRACT VALUE:        $     200,000.00


In [101]:
profit_per_mmbtu = result['contract_value'] / result['total_volume']
profit_margin = (result['contract_value'] / result['total_purchase_cost']) * 100

In [102]:
print(f"\nKEY METRICS:")
print(f"  Total Volume: {result['total_volume']:,.0f} MMBtu")
print(f"  Profit per MMBtu: ${profit_per_mmbtu:.4f}")
print(f"  Profit Margin: {profit_margin:.2f}%")


KEY METRICS:
  Total Volume: 1,000,000 MMBtu
  Profit per MMBtu: $0.2000
  Profit Margin: 1.80%


In [103]:
print(" MODEL COMPLETE AND READY FOR PRODUCTION!")
print("="*70)
print("\nModel Features:")
print("  ✓ Handles multiple injection/withdrawal dates")
print("  ✓ Uses real historical price data")
print("  ✓ Includes all cost components")
print("  ✓ Provides detailed breakdown")
print("  ✓ Ready for trading desk usage")

 MODEL COMPLETE AND READY FOR PRODUCTION!

Model Features:
  ✓ Handles multiple injection/withdrawal dates
  ✓ Uses real historical price data
  ✓ Includes all cost components
  ✓ Provides detailed breakdown
  ✓ Ready for trading desk usage
